In [300]:
import warnings
warnings.filterwarnings('ignore')

In [295]:
import datetime
import pandas as pd
import numpy as np

In [296]:
all_data = pd.read_csv("data.csv")
all_data.head() 

,Row,Participant,Phase,Day,Date,MET_Session,Exercise_Duration_Total,Exercise_METmin_Total,%Complete_total,Mean_total,...,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,1,1,1,1,12/2/19,0,0,0.0,99.30555556,114.472028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2,12/3/19,0,0,0.0,100,132.28125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,3,12/4/19,0,0,0.0,100,141.96875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1,4,12/5/19,0,0,0.0,82.63888889,108.0840336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,1,5,12/6/19,0,0,0.0,97.56944444,126.13879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
df = all_data[["Participant","Date","Phase","%Complete_total","Exercise_Duration_Total", "Exercise_METmin_Total", "Planned Day Off_binary", "Not Enough Time_binary", 
               "Lacked Positive Feelings_binary", "Had Negative Feelings_binary", "Lacked Energy_binary", 
              "Felt Fatigued_binary", "Blood sugar management issue_binary", "Sick", "Morning Fear of Hypoglycemia",
              "Evening Fear of Hypoglycemia", "Sleep Quality", "FearOfHypoglycemia", "Felt Fatigued", "Had Negative Feelings"]]
df

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,Lacked Energy_binary,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings
0,1,12/2/19,1,99.30555556,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
1,1,12/3/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
2,1,12/4/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
3,1,12/5/19,1,82.63888889,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
4,1,12/6/19,1,97.56944444,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,23,11/7/20,3,99.30555556,60,120.0,N/.A,N/.A,N/.A,N/.A,N/.A,N/.A,N/.A,0,0,1,5,N/.A,NaN,NaN
1961,23,11/8/20,3,45.83333333,0,0.0,0,0,0,0,1,1,0,0,3,0,5,99,1,99
1962,23,11/9/20,3,82.98611111,0,0.0,0,1,0,0,0,0,0,0,0,0,4,99,99,99
1963,23,11/10/20,3,100,0,0.0,0,1,0,0,0,0,0,0,0,0,5,99,99,99


In [306]:
# Read and combine data
def read_and_combine_dexcom_with_parts(participantNumber, high=400, low=40, key="Glucose Value (mg/dL)"):
    patientGlucoseDf_1 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_1.csv")
    patientGlucoseDf_2 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_2.csv")
    patientGlucoseDf = pd.concat([patientGlucoseDf_1, patientGlucoseDf_2])
    patientGlucoseDf = patientGlucoseDf.dropna(subset=[key])
    patientGlucoseDf[key].replace({"High": high, "Low": low}, inplace=True)
    patientGlucoseDf[key] = patientGlucoseDf[key].astype(float)
    return patientGlucoseDf

# Patient One Read Data and Combine
def read_and_combine_carelink_with_parts(participantNumber):
    patientGlucoseDf_1 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_1.csv")
    patientGlucoseDf_2 = pd.read_csv("glucose_data/patient_"+str(participantNumber)+"_2.csv")
    patientGlucoseDf_1 = patientGlucoseDf_1.drop(['patient_1_1'], axis=1)
    patientGlucoseDf_2 = patientGlucoseDf_2.drop(['Index'], axis=1)
    patientGlucoseDf = pd.concat([patientGlucoseDf_1, patientGlucoseDf_2])
    return patientGlucoseDf.dropna(subset=['Sensor Glucose (mg/dL)'])

# fix time and sort by time
def fix_time(patientGlucoseDf):
    dates = []
    for timestamp in patientGlucoseDf["Timestamp (YYYY-MM-DDThh:mm:ss)"]:
        d = datetime.datetime.strptime(timestamp[0:10], '%Y-%m-%d')
        just_day = datetime.datetime.strftime(d, '%d').lstrip('0')
        just_month = datetime.datetime.strftime(d, '%m').lstrip('0')
        just_year = datetime.datetime.strftime(d, '%y')
        dates.append(just_month+'/'+just_day+'/'+just_year)
    patientGlucoseDf["Date"] = dates
    return patientGlucoseDf

# Combine Data on a day level
def agreg_on_day_level(key, patientGlucoseDf):
    
    patientGlucoseDf = patientGlucoseDf[['Date', key]]
    aggregation_functions = {'Date': 'first', key: 'mean'}
    return patientGlucoseDf.groupby(patientGlucoseDf['Date']).aggregate(aggregation_functions)

# Search for corresponding data and return patient dataframe with glucose as column
def search_for_corresponding_data(participantNumber, patientGlucoseDf, key):
    df_patient = df.loc[df['Participant'] == participantNumber]
    glucose_values = []
    for date in list(df_patient['Date']):
        temp = patientGlucoseDf.loc[patientGlucoseDf['Date'] == date]
        if temp.shape[0] == 1:
            val = temp[key].values[0]
            glucose_values.append(val)
        else:    
            glucose_values.append(np.nan)
    df_patient["Sensor Glucose (mg/dL)"] = glucose_values
    df_patient = df_patient.loc[df_patient['%Complete_total'].astype(float) > 70]	
    return df_patient

In [304]:
# Carelink Data PreProcessing
patient_1 = read_and_combine_carelink_with_parts(1)
patient_1 = agreg_on_day_level('Sensor Glucose (mg/dL)', patient_1)
df_patient_1 = search_for_corresponding_data(1, patient_1, 'Sensor Glucose (mg/dL)')
df_patient_1

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
0,1,12/2/19,1,99.30555556,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),115.125874
1,1,12/3/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),134.065972
2,1,12/4/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),130.659722
3,1,12/5/19,1,82.63888889,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),126.756944
4,1,12/6/19,1,97.56944444,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),118.207792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1,3/17/20,3,97.2,0,0.0,0,1,0,0,...,0,0,0,.,0,9,99,99,99,100.367347
93,1,3/18/20,3,96.5,0,0.0,0,0,0,1,...,0,0,0,0,0,5,99,99,1,152.575540
94,1,3/19/20,3,94.8,0,0.0,0,0,0,0,...,0,0,0,0,0,7,99,99,99,141.222222
95,1,3/20/20,3,92.0,0,0.0,.,.,.,.,...,.,.,.,.,.,.,.,.,.,125.584000


In [307]:
# Dexcom Data Preprocessing
patient_2 = read_and_combine_dexcom_with_parts(2)
patient_2 = fix_time(patient_2)
patient_2 = agreg_on_day_level('Glucose Value (mg/dL)', patient_2)
df_patient_2 = search_for_corresponding_data(2, patient_2, 'Glucose Value (mg/dL)')
df_patient_2

,Participant,Date,Phase,%Complete_total,Exercise_Duration_Total,Exercise_METmin_Total,Planned Day Off_binary,Not Enough Time_binary,Lacked Positive Feelings_binary,Had Negative Feelings_binary,...,Felt Fatigued_binary,Blood sugar management issue_binary,Sick,Morning Fear of Hypoglycemia,Evening Fear of Hypoglycemia,Sleep Quality,FearOfHypoglycemia,Felt Fatigued,Had Negative Feelings,Sensor Glucose (mg/dL)
98,2,12/3/19,1,93.75,30,315.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),155.024306
99,2,12/4/19,1,70.48611111,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),148.263941
100,2,12/5/19,1,81.25,30,315.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),113.700000
101,2,12/6/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),126.982639
102,2,12/7/19,1,100,0,0.0,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),...,No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),No diaries (baseline),108.881944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,2,3/11/20,3,96.52777778,0,0.0,0,1,0,0,...,1,0,0,0,0,9,99,2,99,189.118056
192,2,3/12/20,3,100,0,0.0,.,.,.,.,...,.,.,.,1,.,.,.,.,.,218.253472
193,2,3/13/20,3,100,0,0.0,0,0,0,0,...,0,0,0,.,0,8,99,99,99,184.350694
194,2,3/14/20,3,100,60,420.0,N/.A,N/.A,N/.A,N/.A,...,N/.A,N/.A,0,0,0,7,N/.A,NaN,NaN,183.927083


In [ ]:
# Carelink Data PreProcessing
patient_4 = read_and_combine_carelink_with_parts(4)
patient_4 = agreg_on_day_level('Sensor Glucose (mg/dL)', patient_4)
df_patient_4 = search_for_corresponding_data(4, patient_1, 'Sensor Glucose (mg/dL)')
df_patient_4